In [1]:
import torch
import pickle
import argparse
import numpy as np
import torch.nn as nn
from torchmetrics import Accuracy
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
import os
from fastai.vision.all import untar_data, URLs
from dime.utils import MaskLayer2d
from dime import CMIEstimator
from pytorch_lightning import Trainer
from dime.masking_pretrainer import MaskingPretrainer
from dime.resnet_imagenet import resnet18, Predictor, ValueNetwork, ResNet18Backbone, resnet50
from dime.vit import PredictorViT, ValueNetworkViT
import timm
import matplotlib.pyplot as plt

/homes/gws/sgadgil/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Get Dataset

In [2]:
acc_metric = Accuracy(task='multiclass', num_classes=100)

image_size = 224
mask_width = 14
mask_type = 'zero'
mask_layer = MaskLayer2d(append=False, mask_width=mask_width, patch_size=image_size/mask_width)

device = torch.device('cuda:5')

norm_constants = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

transforms_test = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize(*norm_constants),
    ])

dataset_path = "/projects/leelab2/sgadgil/ImageNet100"
test_dataset = ImageFolder(dataset_path+'/val.X', transforms_test)

# Prepare dataloaders.
mbsize = 32
test_dataloader = DataLoader(test_dataset, batch_size=mbsize, pin_memory=True, num_workers=4)


# Load pretrained model

In [3]:
arch = "vit_small_patch16_224"
path = "/projects/leelab/sgadgil/Projects/DIME/experiments/Imagenet100/logs/max_features_50_vit_small_patch16_224_trial_1/version_1/checkpoints/best_val_perfomance_model.ckpt"

if arch in ['resnet18']:
    backbone, expansion = ResNet18Backbone(eval(arch + '(pretrained=True)'))
    predictor =  Predictor(backbone, expansion, num_classes=100)
    block_layer_stride = 1
    if mask_width == 14:
        block_layer_stride = 0.5
    value_network = ValueNetwork(backbone, expansion, block_layer_stride=block_layer_stride)
else:
    backbone = timm.create_model(arch, pretrained=True)
    predictor =  PredictorViT(backbone, num_classes=100)
    value_network = ValueNetworkViT(backbone)
    
greedy_cmi_estimator = CMIEstimator.load_from_checkpoint(path,
                                                         value_network=value_network,
                                                         predictor=predictor,
                                                         mask_layer=mask_layer,
                                                         lr=1e-5,
                                                         min_lr=1e-8,
                                                         max_features=50,
                                                         eps=0.05,
                                                         loss_fn=nn.CrossEntropyLoss(reduction='none'),
                                                         val_loss_fn=acc_metric,
                                                         eps_decay=0.2,
                                                         eps_steps=10,
                                                         patience=3,
                                                         feature_costs=None)
trainer = Trainer(
                    accelerator='gpu',
                    devices=[device.index],
                    precision=16
                )

Global seed set to 42


Hello


Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


# Evaluate Penalized Policy

In [ ]:
avg_num_features_lamda = []
accuracy_scores_lamda = []
all_masks_lamda =[]

lamda_values = list(np.geomspace(0.005, 0.03, num=12))
for lamda in lamda_values:
    metric_dict = greedy_cmi_estimator.inference(trainer, test_dataloader, feature_costs=None, lam=lamda)
        
    y = metric_dict['y']
    pred = metric_dict['pred']
    accuracy_score = acc_metric(pred, y)
    final_masks = np.array(metric_dict['mask'])
    accuracy_scores_lamda.append(accuracy_score)
    avg_num_features_lamda.append(np.mean(np.sum(final_masks, axis=1)))
#     results_dict['acc'][np.mean(np.sum(final_masks, axis=1))] = accuracy_score

    print(f"Lambda={lamda}, Acc={accuracy_score}, Avg. num features={np.mean(np.sum(final_masks, axis=1))}")
    all_masks_lamda.append(final_masks)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Predicting DataLoader 0: 100%|████████████████| 157/157 [11:39<00:00,  4.45s/it]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Lambda=0.005, Acc=0.8636000156402588, Avg. num features=43.07740020751953
Predicting DataLoader 0: 100%|████████████████| 157/157 [11:17<00:00,  4.31s/it]
Lambda=0.005884519781214263, Acc=0.8605999946594238, Avg. num features=40.370399475097656


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


...

Predicting DataLoader 0: 100%|████████████████| 157/157 [09:59<00:00,  3.82s/it]
Lambda=0.009592550525713004, Acc=0.848800003528595, Avg. num features=31.927799224853516


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Predicting DataLoader 0: 100%|████████████████| 157/157 [09:32<00:00,  3.65s/it]
Lambda=0.011289510664171094, Acc=0.8442000150680542, Avg. num features=29.156600952148438


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Predicting DataLoader 0:  69%|███████████     | 109/157 [06:25<02:49,  3.54s/it]

# Evaluate Budget Constrained Policy

In [4]:
avg_num_features_budget = []
accuracy_scores_budget = []
all_masks_budget=[]

max_budget_values = [1.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0]
for budget in max_budget_values:
    metric_dict_budget = greedy_cmi_estimator.inference(trainer, test_dataloader, feature_costs=None, budget=budget)
        
    y = metric_dict_budget['y']
    pred = metric_dict_budget['pred']
    accuracy_score = acc_metric(pred, y)
    final_masks = np.array(metric_dict_budget['mask'])
    accuracy_scores_budget.append(accuracy_score.numpy().item())
    avg_num_features_budget.append(np.mean(np.sum(final_masks, axis=1)))
#     results_dict['acc'][np.mean(np.sum(final_masks, axis=1))] = accuracy_score
    print(f"Budget={budget}, Acc={accuracy_score}, Avg. num features={np.mean(np.sum(final_masks, axis=1))}")

    all_masks_budget.append(final_masks)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Predicting DataLoader 0: 100%|████████████████| 157/157 [00:24<00:00,  6.33it/s]
Budget=1.0, Acc=0.21140000224113464, Avg. num features=1.0


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Predicting DataLoader 0: 100%|████████████████| 157/157 [00:29<00:00,  5.33it/s]
Budget=5.0, Acc=0.5645999908447266, Avg. num features=5.0


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


...

Predicting DataLoader 0: 100%|████████████████| 157/157 [01:38<00:00,  1.59it/s]
Budget=20.0, Acc=0.7820000052452087, Avg. num features=20.0


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Predicting DataLoader 0: 100%|████████████████| 157/157 [02:01<00:00,  1.29it/s]
Budget=25.0, Acc=0.7990000247955322, Avg. num features=25.0


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Predicting DataLoader 0: 100%|████████████████| 157/157 [02:24<00:00,  1.08it/s]
Budget=30.0, Acc=0.8105999827384949, Avg. num features=30.0


In [5]:
accuracy_scores_budget

[tensor(0.2114),
 tensor(0.5646),
 tensor(0.7120),
 tensor(0.7628),
 tensor(0.7820),
 tensor(0.7990),
 tensor(0.8106)]

# Evaluate Confidence Constrained Policy

In [ ]:
avg_num_features_confidence = []
accuracy_scores_confidence = []
all_masks_confidence=[]
confidence_values = list(np.arange(0.1, 1, 0.1))

for confidence in confidence_values:
    metric_dict = greedy_cmi_estimator.inference(trainer, test_dataloader, feature_costs=None, confidence=confidence)
    y = metric_dict['y']
    pred = metric_dict'pred']
    accuracy_score = acc_metric(pred, y)

    final_masks = np.array(metric_dict['mask'])
    accuracy_scores_confidence.append(accuracy_score)
    avg_num_features_confidence.append(np.mean(np.sum(final_masks, axis=1)))
    print(f"Confidence={confidence}, Acc={accuracy_score}, Avg. num features={np.mean(np.sum(final_masks, axis=1))}")
    all_masks_confidence.append(final_masks)